In [61]:
#Description : This is a sentiment analysis program that parses the tweets fetched from Twitter using Python

In [62]:
# Import the libraries
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [63]:
# Load the data
from google.colab import files
uploaded = files.upload()

Saving creds.csv to creds (1).csv


In [64]:
#Get the data
log = pd.read_csv('creds.csv')
print(log)

                                                  Key  Unnamed: 1
0                           yDmtZi6aKQYRqg4U1X7P25PkG         NaN
1   Bjs5nW6l4zT4BhiKJPwVSN8jbUHiwgwimkd5pe1RdND9ki...         NaN
2   1544693349030010881-S9VavYOZadgqQBR5rUuyDTdxEY...         NaN
3       9iTYT2bZPYaCVsVsQmmi6wSdmTl8BAvyU44YH9zJRyDyS         NaN
4                                                 NaN         NaN
5                                                 NaN         NaN
6                                                 NaN         NaN
7                                                 NaN         NaN
8                                                 NaN         NaN
9                                                 NaN         NaN
10                                                NaN         NaN
11                                                NaN         NaN
12                                                NaN         NaN


In [65]:
# Twitter API credentials
consumerKey = log['Key'][0]
consumerSecret = log['Key'][1]
accessToken = log['Key'][2]
accessTokenSecret = log['Key'][3]

print(consumerKey)

yDmtZi6aKQYRqg4U1X7P25PkG


In [66]:
# Create the authentication object
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)

# Set the access token and access token secret
authenticate.set_access_token(accessToken, accessTokenSecret)

# Create the API object while passing in the auth information
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [69]:
# Extract 100 tweets from the twitter user
posts = api.user_timeline(screen_name = "BillGates", count = 100, lang = "en", tweet_mode = "extended")

# Print the last 5 tweets from the account
print("Show the 5 recent tweets: \n")
i = 1
for tweet in posts[0:5]:
  print(str(i) + ')' + tweet.full_text + '\n')
  i += 1

In [ ]:
#Create a dataframe with a column called Tweets
df = pd.DataFrame( [tweet.full_text for tweet in posts] , columns = ['Tweets'])

# Show the first 5 rows of data
df.head()

In [ ]:
#Clean the text

# Create a function to clean the tweets
def cleanTxt(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) #Removed @mentions
  text = re.sub(r'#', '', text) # Removing the '#' symbol
  text = re.sub(r'RT[\s]+', '', text) #Removing RT
  text = re.sub(r'https?:\/\/\S+', '', text) # Remove the hyper link

  return text

# Cleaning the text
df['Tweets'] = df['Tweets'].apply(cleanTxt)

#Show the cleaned text
df

In [ ]:
# Create a function to get the sujectivity
def getSubjectivity(text):
  return TextBlob(text).sentiment.getSubjectivity

# Create a function to get the polarity
def getPolarity(text):
  return TextBlob(text).sentiment.polarity

# Create two new Columns
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

# Show the new dataframe with the new columns
df

In [ ]:
# Plot The Word Cloud
allWords = ' '.join( [twts for twts in df['Tweets']])
wordCloud = WordCloud(width = 500, height = 300m random_state = 21, max_font_size = 110).generate(allWords)

plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Create a function to compute the negative, neutral and positive analysis
def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'

df['Analysis'] = df['Polarity'].apply(getAnalysis)

#Show the dataframe
df

In [ ]:
# Print all of the positive tweets
j = 1
sortedDF = df.sort_values(by=['Polarity'])
for in in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == 'Positive'):
    print(str(j) + ') ' + sortedDF['Tweets'][i])
    print()
    j += 1

In [ ]:
# Print the negative tweets

 j = 1
 sortedDF = df.sort_values(by=['Polarity'], ascending = 'False')
 for i in range(0, sortedDF.shape[0]):
   if(sortedDF['Analysis'][i] == 'Negative'):
    print(str(j) + ') ' + sortedDF['Tweets'][i])
    j += 1

In [ ]:
# Plot the polarity and subjectivity
plt.figure(figsize = (8, 6))
for i in range(0, df.shape[0]):
  plt.scatter(df['Polarity'][i], df['Subjectivity'][i], color='Blue')

plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

In [ ]:
# Get the percentage of positive tweets
ptweets = df[df.Analysis == 'Positive']
ptweets = ptweets['Tweets']

round( (ptweets.shape[0] / df.shape[0]) * 100, 1)

In [ ]:
# Get the percentage of negative tweets
ntweets = df[df.Analysis == 'Negative']
ntweets = ntweets['Tweets']

round( (ntweets.shape[0] / df.shape[0] * 100), 1)

In [ ]:
# Show the value counts
df['Analysis'].value_counts()

# Plot and visualize the counts
plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis'].value_counts().plot(kind='bar')
plt.show